This is just a script for generating VM steps for a beamer

In [10]:
bytecode = '''
  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               2 (a0)

  3           4 LOAD_CONST               2 (1)
              6 STORE_FAST               3 (a1)

  4           8 LOAD_CONST               3 (2)
             10 STORE_FAST               4 (a2)

  5          12 LOAD_FAST                2 (a0)
             14 LOAD_FAST                3 (a1)
             16 BINARY_ADD
             18 LOAD_FAST                4 (a2)
             20 BINARY_ADD
             22 STORE_FAST               5 (a3)

  6          24 LOAD_FAST                0 (a)
             26 LOAD_METHOD              0 (sum)
             28 CALL_METHOD              0
             30 LOAD_CONST               1 (0)
             32 COMPARE_OP               4 (>)
             34 POP_JUMP_IF_FALSE       48

  7          36 LOAD_FAST                0 (a)
             38 LOAD_FAST                1 (b)
             40 BINARY_ADD
             42 LOAD_FAST                5 (a3)
             44 BINARY_ADD
             46 RETURN_VALUE

  8     >>   48 LOAD_FAST                0 (a)
             50 LOAD_FAST                1 (b)
             52 BINARY_SUBTRACT
             54 RETURN_VALUE
'''.strip()

In [11]:
def get_hightlight_lineno():
    for no,line in enumerate(bytecode.split('\n')):
        line = line.strip()
        if not line: continue
        fs = line.split()
        instr_no = -1

        if len(fs) == 5:
            instr_no = fs[1]
        elif len(fs) == 4:
            instr_no = fs[0]
        elif len(fs) == 6:
            instr_no = fs[2]
        elif len(fs) == 2:
            instr_no = fs[0]
        elif len(fs) == 3:
            instr_no = fs[0]
        else:
            print(fs)
            raise

        yield instr_no, no

In [12]:
from dataclasses import dataclass
from typing import *

@dataclass
class Record:
    highlight_no:int
    instr_no: str
    stack: List[str]
    annotation: List[str]
    subgraph: List[str]
    step: int

In [13]:
def get_block(record:Record):
    return """
    ** Tracing on a simple case (step {step}) :B_block:
:PROPERTIES:
:BEAMER_env: block
:BEAMER_opt: fragile
:END:
*** C :B_columns:
:PROPERTIES:
:BEAMER_opt: t
:BEAMER_env: columns
:END:
**** Tracing the bytecode :BMCOL:
:PROPERTIES:
:BEAMER_col: 0.5
:END:

\\tiny
#+BEGIN_SRC latex
\\begin{{minted}}[highlightlines={highlight_no}, highlightcolor=pink]{{python}}
  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               2 (a0)

  3           4 LOAD_CONST               2 (1)
              6 STORE_FAST               3 (a1)

  4           8 LOAD_CONST               3 (2)
             10 STORE_FAST               4 (a2)

  5          12 LOAD_FAST                2 (a0)
             14 LOAD_FAST                3 (a1)
             16 BINARY_ADD
             18 LOAD_FAST                4 (a2)
             20 BINARY_ADD
             22 STORE_FAST               5 (a3)

  6          24 LOAD_FAST                0 (a)
             26 LOAD_METHOD              0 (sum)
             28 CALL_METHOD              0
             30 LOAD_CONST               1 (0)
             32 COMPARE_OP               4 (>)
             34 POP_JUMP_IF_FALSE       48

  7          36 LOAD_FAST                0 (a)
             38 LOAD_FAST                1 (b)
             40 BINARY_ADD
             42 LOAD_FAST                5 (a3)
             44 BINARY_ADD
             46 RETURN_VALUE

  8     >>   48 LOAD_FAST                0 (a)
             50 LOAD_FAST                1 (b)
             52 BINARY_SUBTRACT
             54 RETURN_VALUE
\\end{{minted}}
#+END_SRC


**** States :B_block:BMCOL:
:PROPERTIES:
:BEAMER_col: 0.5
:BEAMER_opt: t
:BEAMER_env: block
:END:

{annotation}

***** Instruction counter
\small

{instr_no}



***** Frame stack
:PROPERTIES:
:BEAMER_env: block
:END:
\small
|---+--------|
| id | values |
|---+--------|
{stack}



***** Output graph :B_block:
:PROPERTIES:
:BEAMER_env: block
:END:
\\tiny

|---+-----|
|  id | ops |
|---+-----|
{subgraph}



""".format(
    step=record.step,
    highlight_no=record.highlight_no+1,
    annotation='\n'.join(record.annotation) if type(record.annotation) == list else record.annotation,
instr_no=record.instr_no,
stack = '\n'.join(["| %d | ~%s~ |" % (no,item) for no, item in enumerate(record.stack)]),
subgraph = '\n'.join(["| %d | ~%s~ |" % (no,item) for no, item in enumerate(record.subgraph)]),
).lstrip()


In [14]:
steps = [x for x in get_hightlight_lineno()]
print(steps)
len(steps)
#get_hightlight_lineno()

[('0', 0), ('2', 1), ('4', 3), ('6', 4), ('8', 6), ('10', 7), ('12', 9), ('14', 10), ('16', 11), ('18', 12), ('20', 13), ('22', 14), ('24', 16), ('26', 17), ('28', 18), ('30', 19), ('32', 20), ('34', 21), ('36', 23), ('38', 24), ('40', 25), ('42', 26), ('44', 27), ('46', 28), ('48', 30), ('50', 31), ('52', 32), ('54', 33)]


28

In [15]:
records = []
step = 0

# a0 = 0
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(0)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]",
              ],
    annotation="Load a constant, push it on the stack",
))
step += 1
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=[],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Store to local variable a0",
))
step += 1

# a1 = 1
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(1)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Load a constant, push it on the stack",
))
step += 1
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=[],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Store to local variable a1",
))
step += 1


# a2 = 2
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(2)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Load a constant, push it on the stack",
))
step += 1
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=[],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Store to local variable a2",
))
step += 1

In [16]:
# a3 = a0 + a1 + a2
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(a0)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Load a constant, push it on the stack",
))
step += 1
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(a0)', 'ConstantValue(a1)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Load a constant, push it on the stack",
))
step += 1
records.append(Record( # BINARY_ADD
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(1)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="",
))
step += 1

records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(1)', 'ConstantValue(a2)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Load a constant, push it on the stack",
))
step += 1

records.append(Record( # BINARY_ADD
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['ConstantValue(3)',],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="",
))
step += 1

# STORE_FAST a3
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=[],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="",
))
step += 1


In [17]:
# LOAD_FAST (a)
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['TensorVariable(a)',],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="Load a tensor, push to stack",
))
step += 1

# LOAD_METHOD
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['TensorVariable(a)', 'GetAttrVariable(TensorVariable(), sum)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", ],
    annotation="",
))
step += 1


# CALL_METHOD
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['TensorVariable()'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", 
              "%sum_1 : [#users=0] = call_method[target=sum](args = (%a,), kwargs = {})",
              ],
    annotation="",
))
step += 1

# LOAD_CONST
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['TensorVariable()', 'ConstantValue(0)'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", 
              "%sum_1 : [#users=0] = call_method[target=sum](args = (%a,), kwargs = {})",
              ],
    annotation="",
))
step += 1


# COMPARE_OP
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=['TensorVariable()'],
    subgraph=["%a : torch.Tensor [#users=0] = placeholder[target=a]",
              "%b : torch.Tensor [#users=0] = placeholder[target=b]", 
              "%sum_1 : [#users=0] = call_method[target=sum](args = (%a,), kwargs = {})",
              "%gt : [#users=0] = call_method[target=gt](args = (%sum_1, 0), kwargs = {})",
              ],
    annotation="",
))
step += 1


# JUMP_IF_FALSE
records.append(Record(
    highlight_no=steps[step][1],
    instr_no=steps[step][0],
    step=step,
    stack=[],
    subgraph=[ ],
    annotation=["TensorVariable as condition, trigger compile subgraph",
                "Follows the heuristics of ~generic_jump()~"],
))
step += 1





In [18]:
for record in records:
    print(get_block(record))

** Tracing on a simple case (step 0) :B_block:
:PROPERTIES:
:BEAMER_env: block
:BEAMER_opt: fragile
:END:
*** C :B_columns:
:PROPERTIES:
:BEAMER_opt: t
:BEAMER_env: columns
:END:
**** Tracing the bytecode :BMCOL:
:PROPERTIES:
:BEAMER_col: 0.5
:END:

\tiny
#+BEGIN_SRC latex
\begin{minted}[highlightlines=1, highlightcolor=pink]{python}
  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               2 (a0)

  3           4 LOAD_CONST               2 (1)
              6 STORE_FAST               3 (a1)

  4           8 LOAD_CONST               3 (2)
             10 STORE_FAST               4 (a2)

  5          12 LOAD_FAST                2 (a0)
             14 LOAD_FAST                3 (a1)
             16 BINARY_ADD
             18 LOAD_FAST                4 (a2)
             20 BINARY_ADD
             22 STORE_FAST               5 (a3)

  6          24 LOAD_FAST                0 (a)
             26 LOAD_METHOD              0 (sum)
             28 CALL_METHOD      